In [1]:
import pandas as pd
import kaggle# to access kaggle
import pandas as pd
from pathlib import Path
import csv
import zipfile# importing dependence for zipfile
from kaggle.api.kaggle_api_extended import KaggleApi


In [2]:
#loading filess
# File to Load (Remember to Change These)
api = KaggleApi()
api.authenticate()
#downloading files
api.dataset_download_file("umeradnaan/tourism-dataset",
                          file_name="tourism_dataset.csv")#path="./data 

api.dataset_download_file("jessemostipak/hotel-booking-demand",
                          file_name="hotel_bookings.csv")


Dataset URL: https://www.kaggle.com/datasets/umeradnaan/tourism-dataset
Dataset URL: https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand


True

In [3]:
#unziping file
with zipfile.ZipFile("hotel_bookings.csv.zip", "r") as zipref:
    zipref.extractall()


In [5]:
#reading files
hotel_data = Path("hotel_bookings.csv")
travel_data = Path("tourism_dataset.csv")
hotel = pd.read_csv(hotel_data)
travel = pd.read_csv(travel_data)
print(hotel.keys())
print(travel.keys())


Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')
Index(['Location', 'Country', 'Category', 'Visitors', 'Rating', 'Revenue',
       'Accommodation_Available'],
      dtype='object')


In [ ]:
travel_data = pd.DataFrame({
    "Country": data["country"]
    ""
})

In [ ]:
data["country"]

    